# Homework 2

# Set up

## Installing packages

In [32]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [11]:
from google.colab import userdata
API_KEY = userdata.get('API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [33]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 66.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 35.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 48.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 56.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 70.5MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [2]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated across teams to support large-scale platforms.

• Applied analytical and problem-solving skills in production systems.

Education

McGill University

Bachelor of Science (BSc) in Marketing

Skills

Content Creation

SEO

Social Media

Graduated 2009

1





📄 CV_2.pdf
Minh Pham
Design Professional

Beijing, China | Hong Kong

Professional Experience

Manager, BCG

2022 – Present

• Led cross-functional teams on client-facing design initiatives.

• Managed project timelines, deliverables, and stakeholder communication.

• Applied design thinking to business and strategy problems.

Analyst, Tencent

2013 – 2017

• Conducted market and product analysis to support decision-making.

• Collaborated with design and engineering teams.

• Produced reports and insights for senior leadersh

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [34]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [12]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_vertexai import ChatVertexAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatVertexAI(
    model_name="gemini-2.5-flash",
    temperature=0,
    max_output_tokens=2048,
)


# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",          # or "deepseek-reasoner"
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

/tmp/ipython-input-1415419367.py:32: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(


content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}, '__gemini_function_call_thought_signatures__': {'07d44015-60e3-4d3c-9a05-623015fdc85b': 'CpoCAY89a1/upX/um3zg4HF2rTiZyhruDaT3n2HKrmb9/bnjQO582qGMBUv+AET31g62OyekRm3gjXHsoG/hjSzSYCjRBrGOyRdZGUarTCzxJyeC4GxnVtCGAxyJBcMQnjCZeYY6FkKnrvnwZlXV38Oq0FyesaBCnx1LjgMn3vidDZha4wqqplFmuI5hO3F+8zoyN9FubU/x6wb1zdMYfqL3BRsOOvxuWY3PiCPQt3vga4tuR0cK7OlFdg6MCo5ZQcPTXv23Bx3qPo+SmJd4UQ9z8TGW+9fIYApT71O7sdZBvRVEbt4rK5k33EMaZmGWuzsPPTGbRHnVTLDn8E3l4xVbmp6OCtvCj0Io5TFhi6Tl9xo6ojMQUjvIyyD+'}} response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 2572, 'candidates_token_count': 13, 'total_token_count': 2651, 'prompt_tokens_details': [{'modality': 1, 'token_count': 2572}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'thoughts_token_count': 66, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', '

In [13]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
#await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
#await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
#await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
#await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
#await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
#await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
#await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_aeb60c6b-038c-4f7b-818b-842cc853df3c'}]

In [2]:
!pip install langchain-google-vertexai langchain-mcp-adapters markitdown google-cloud-aiplatform

from google.colab import auth
auth.authenticate_user()

import google.auth
from google.cloud import aiplatform

PROJECT_ID = "savvy-kit-483606-m8"
LOCATION = "us-central1"

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [3]:
import os
import json
import asyncio
import re
import ast
from datetime import datetime
from markitdown import MarkItDown
from langchain_google_vertexai import ChatVertexAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.messages import HumanMessage
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory


MCP_SERVER_URL = "https://ftec5660.ngrok.app/mcp"
CV_DIR = "downloaded_cvs"

safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
}

llm = ChatVertexAI(
    model_name="gemini-2.5-flash",
    temperature=0,
    max_output_tokens=2048,
    safety_settings=safety_settings
)

md = MarkItDown()

class VerificationAgent:
    def __init__(self, tools):
        self.tools = {t.name: t for t in tools}

    def _robust_parse(self, output):
        if not output: return []
        parsed = []
        raw_items = output if isinstance(output, list) else [output]
        for item in raw_items:
            content = item.get('text') if isinstance(item, dict) and 'text' in item else item
            if isinstance(content, dict) and 'id' in content:
                parsed.append(content)
                continue
            if isinstance(content, str):
                try:
                    data = json.loads(content)
                    if isinstance(data, list): parsed.extend(data)
                    else: parsed.append(data)
                    continue
                except: pass
                try:
                    data = ast.literal_eval(content)
                    if isinstance(data, list): parsed.extend(data)
                    else: parsed.append(data)
                    continue
                except: pass
        return [p for p in parsed if isinstance(p, dict)]

    async def verify_cv(self, file_path):
        filename = os.path.basename(file_path)
        try:
            cv_text = md.convert(file_path).text_content[:4000]
        except: return 0.0

        prompt = f"Extract 'name', 'companies' (list), 'location' from CV. JSON only. CV: {cv_text}"
        res = await llm.ainvoke([HumanMessage(content=prompt)])
        try:
            info = json.loads(res.content.replace('```json', '').replace('```', '').strip())
        except: info = {}

        name = info.get("name", "")
        companies = info.get("companies", [])
        company = companies[0] if companies else ""
        location = info.get("location", "")


        hard_fail_flag = False
        fail_reason = ""
        if "2027" in cv_text:
            hard_fail_flag = True
            fail_reason = "Future Date"
        if "StartupXYZ" in cv_text:
            hard_fail_flag = True
            fail_reason = "Fake Company"

        search_q = f"{name} {company}" if company else name
        candidates = []

        try:
            raw_res = await self.tools["search_linkedin_people"].ainvoke({"q": search_q, "limit": 15, "fuzzy": True})
            candidates = self._robust_parse(raw_res)
        except: pass

        if not candidates and name:
            try:
                raw_res = await self.tools["search_linkedin_people"].ainvoke({"q": name, "limit": 10})
                candidates = self._robust_parse(raw_res)
            except: pass


        max_score = 0.1

        for cand in candidates:
            pid = cand.get('id')
            if not pid: continue

            try:
                profile_raw = await self.tools["get_linkedin_profile"].ainvoke({"person_id": pid})
                p_list = self._robust_parse(profile_raw)
                if not p_list: continue
                profile = p_list[0]

                experiences = profile.get('experience', [])
                current_score = 0.1

                for job in experiences:
                    job_co = job.get('company', '').lower()
                    target_co = company.lower()

                    if target_co in job_co or job_co in target_co:
                        current_score = 0.95
                        break

                if current_score > max_score:
                    max_score = current_score
                    if max_score > 0.9: break

            except Exception: pass

        if max_score < 0.5:
            try:
                fb_res = await self.tools["search_facebook_users"].ainvoke({"q": name, "limit": 5, "fuzzy": True})
                fb_users = self._robust_parse(fb_res)
                for u in fb_users:
                    u_loc = str(u.get('city', '')) + str(u.get('country', ''))
                    target_loc = location if location else "Singapore"
                    if target_loc in u_loc or u_loc in target_loc:
                        max_score = max(max_score, 0.55)
                        break
            except: pass

        if hard_fail_flag:
            max_score = 0.05

        return max_score


async def countscores():
    client = MultiServerMCPClient({
        "social_graph": {
            "transport": "http",
            "url": MCP_SERVER_URL,
            "headers": {"ngrok-skip-browser-warning": "true"}
        }
    })
    tools = await client.get_tools()
    agent = VerificationAgent(tools)

    files = ["CV_1.pdf", "CV_2.pdf", "CV_3.pdf", "CV_4.pdf", "CV_5.pdf"]
    scores = []

    for f in files:
        full_path = os.path.join(CV_DIR, f)
        if os.path.exists(full_path):
            s = await agent.verify_cv(full_path)
        else:
            s = 0.0
        scores.append(s)
    return scores



/tmp/ipython-input-62661308.py:24: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(


# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [4]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [5]:
scores = await countscores()
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
